# Genetic Programming
## using Python

The following Python code is nothing special in terms of instructions used, but its implications are interesting, for reasons you will understand if you read this piece. 

Near the beginning of our file, we have import statements. These are system libraries, meaning it's what comes with Python3.

In [1]:
import random
from math import floor
import sys
import string

Here, we define a Gene class. 

The Gene exists as a collection, and can mate with another gene, in this case meaning a randomly chosen split between two Gene's contents before random recombination with other genes in the pool. 
A Gene can also undergo mutation, by invoking its mutate function which requires a rate of mutation, think of this as a reading from the bequerel meter. Radiation sends particles around it, hitting atoms and molecules, and sometimes knocking some of them loose, the more radiation (here called "percent mutation"), the less recognizable the gene from before the mutation. 

In [2]:
class Gene():
    def __init__(self, code):
        self.code = code
        self.cost = 9999

    def mate(self, gene):
        middle = int(floor(len(self.code)/2))
        return [Gene(self.code[:middle] + gene.code[middle:]),
                Gene(gene.code[:middle] + self.code[middle:])]

    def mutate(self, rate):
        if random.random() < rate:
            return

        code = ''
        index = round(random.random() * len(self.code))
        for i in range(len(self.code)):
            upOrDown = -1 if round(random.random()) else 1
            if i == index and ord(self.code[i]) + upOrDown < 256 and ord(self.code[i]) > 0:
                code += chr(ord(self.code[i]) + upOrDown)
            else:
                code += self.code[i]

        self.code = code

    def random(self, length):
        code = ''
        for i in range(length):
            code += chr(int(random.random()*255))
        self.code = code

    def calc_cost(self, target):
        total = 0
        
        for i in range(len(self.code)):
            total += (ord(self.code[i]) - ord(target[i])) * (ord(self.code[i]) - ord(target[i]))

        self.cost = total
        

The population acts as a collection for the Genes. It breeds this example of informational genes by sorting by a "cost" or "ranking" function each generation (round), and comparing the highest ranking genetic expression (one of the strings containing) compared to a single desired output string. Basically, this code implements a toy model of evolution. 

In [3]:
class Population():
    def __init__(self, target="Greetings, Worlds!", size=100, log_costs=True):
        self.target = target
        self.members = []
        for i in range(size):
            gene = Gene('')
            gene.random(len(self.target))
            self.members.append(gene)
        self.generationNumber = 0
        
        self.log_costs = log_costs
        if self.log_costs:
            self.cost_log = [] # logs the cost of the highest ranking member
        
    def calc_costs(self):
        for member in self.members:
            member.calc_cost(self.target)

    def mutate(self, chance):
        for member in self.members:
            member.mutate(chance)

    # Should be called after Population.calcCosts()
    def sort(self):
        self.members = sorted(self.members, key=lambda member: member.cost)

    def display(self):
        self.calc_costs()
        self.sort()
        # "Generation", self.generationNumber, self.members[0].cost 
        
        code = self.members[0].code
        clean = ''
        for c in code:
            if c.isprintable():
                clean += c
        print(clean, end='\r')
        sys.stdout.flush()

    def genotype(self, mutation_rate=0.5, display=False):
        while not self._generation(mutation_rate, display):
            pass
            
        if self.log_costs:
            return self.cost_log
        else:
            return self.generationNumber
        
    def _generation(self, mutation_rate=0.5, display=False):
        self.calc_costs()
        self.sort()
        if self.log_costs:
            self.cost_log.append(self.members[0].cost)
        if display:
            self.display()

        children = self.members[0].mate(self.members[1])
        self.members[-2] = children[0]
        self.members[-1] = children[1]

        for member in self.members:
            member.mutate(mutation_rate)
            member.calc_cost(self.target)
            if member.code == self.target:
                self.sort()
                if display:
                    self.display()
                return True

        self.generationNumber += 1
        return False

In [4]:
#targetString = 'As time progresses...'
#
#population = Population(target=targetString, log_costs=False)
#print(population.members[0].code)
#population.genotype(display=True)

In [5]:
def evolve(targetStrings):
    if type(targetStrings) is str:
        strs = targetStrings.split(' ')
        targetStrings = strs
    
    for s in targetStrings:
        population = Population(target=s, log_costs=False)
        population.genotype(display=True)
        c = population.members[0].code
        print(c)

In [6]:
evolve("Welcome to Instantania".split())

Welcome
to
Instantania


## evolve("What's the juciest juice on Earth?")

In [ ]:
evolve("What's the juciest juice on Earth?")

What's
the
juciest
juice
on
Earth?


In [8]:
evolve("Welcome, to the worlds of tomorrow")

Welcome,
to
the
worlds
of
tomorrow
